In [1]:
from datetime import datetime, timedelta
from IPython.display import clear_output
import pandas_datareader.data as web
import yfinance as yf
import pandas as pd
import numpy as np
import pyautogui
import time

pd.options.display.float_format = '{:,.2f}'.format

# Downloads tickers and formatting data

### Fetching tickers from nasdaq and filtering out ETFs are stocks that we have a problem getting their prices

In [2]:
def get_all_tickers(etf=False):
    # Get a list of all available tickers from Yahoo Finance
    all_tickers = web.get_nasdaq_symbols()
    all_tickers = all_tickers[~all_tickers["Test Issue"]]  # Filter out stocks that we can't fetch their data for some reason
    
    if not etf:
        all_tickers = all_tickers[~all_tickers["ETF"]]  # Filter out ETFs
    tickers = all_tickers.index.tolist()
    return [str(ticker) for ticker in tickers if str(ticker).isalpha()]

### Formatting DataFrame

In [3]:
def get_info(df):
    high = df['High'].max()
    close = df['Close'].iloc[-1]
    pct_to_52 = ((high - close) / high) * 100
    volume = df["Volume"].iloc[-1]
    date_of_high = df['High'].idxmax()
    
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "Percent from 52": pct_to_52, "Volume": volume, "doh": date_of_high, "Volume$": volume * close})

    return new_df

### Downloads data

In [4]:
def get_data(tickers_list):
    return yf.download(tickers_list, period="1y")

In [5]:
s = time.time()
data = get_data(get_all_tickers())
data_cop = data.copy()
print(time.time() - s)

[*********************100%***********************]  7411 of 7411 completed

65 Failed downloads:
- ATAKR: Period '1y' is invalid, must be one of ['1d', '5d']
- FMSTW: Period '1y' is invalid, must be one of ['1d', '5d']
- MBTCR: Period '1y' is invalid, must be one of ['1d', '5d']
- FLFVR: Period '1y' is invalid, must be one of ['1d', '5d']
- ASCBR: Period '1y' is invalid, must be one of ['1d', '5d']
- FEXDR: Period '1y' is invalid, must be one of ['1d', '5d']
- SAGAR: Period '1y' is invalid, must be one of ['1d', '5d']
- CLRCR: Period '1y' is invalid, must be one of ['1d', '5d']
- HSPOR: Period '1y' is invalid, must be one of ['1d', '5d']
- MARXR: Period '1y' is invalid, must be one of ['1d', '5d']
- NVACR: Period '1y' is invalid, must be one of ['1d', '5d']
- DISTR: Period '1y' is invalid, must be one of ['1d', '5d']
- BLEUR: Period '1y' is invalid, must be one of ['1d', '5d']
- GECCZ: Period '1y' is invalid, must be one of ['1d', '5d']
- ONFOW: Period '1y' is invalid, must be one of [

In [14]:
df = get_info(data)
cop = df.copy()
df

,Close,52wk High,Percent from 52,Volume,doh,Volume$
A,117.56,160.26,26.64,"735,802.00",2022-12-13,"86,500,881.32"
AA,28.55,58.24,50.98,"2,641,061.00",2022-08-26,"75,389,086.45"
AAC,10.68,10.68,0.05,"88,530.00",2023-08-22,"945,057.77"
AACG,1.23,2.64,53.42,"1,666.00",2022-08-25,"2,048.68"
AACI,10.59,12.20,13.20,"5,005.00",2023-01-30,"53,002.95"
...,...,...,...,...,...,...
ZVSA,0.15,25.00,99.38,"590,733.00",2022-12-12,"90,854.73"
ZWS,28.44,30.87,7.87,"1,038,969.00",2023-07-26,"29,548,278.91"
ZYME,7.88,10.80,27.08,"196,079.00",2023-01-19,"1,544,122.12"
ZYNE,1.29,1.38,6.16,"380,012.00",2023-08-14,"492,115.52"


### Filtering out stocks that doesn't match 

In [15]:
df = cop.copy()
df = df.dropna(subset=['Close'])
vol_thresh = 10000
vold_thresh = 500000
df = df[df['Volume'] >= vol_thresh]
df = df[df['Volume$'] >= vold_thresh]
time_thresh1 = datetime.now() - timedelta(days=90)  # Hit the high between 120 and 200 days ago (3 - 7 months)
time_thresh2 = datetime.now() - timedelta(days=200)  
df = df[(df["doh"] < time_thresh1) & (df["doh"] > time_thresh2)]
df.sort_values(by='Percent from 52', ascending=True).head(15)

,Close,52wk High,Percent from 52,Volume,doh,Volume$
NEWR,85.14,86.00,1.01,"1,334,482.00",2023-05-17,"113,611,127.92"
ARGO,29.70,30.13,1.43,"143,844.00",2023-04-06,"4,272,166.91"
CHKP,132.42,135.93,2.58,"267,750.00",2023-04-12,"35,455,454.51"
LOCC,10.33,10.63,2.82,"351,620.00",2023-04-17,"3,632,234.57"
IVCA,10.74,11.11,3.35,"100,450.00",2023-05-11,"1,078,832.98"
CAE,23.42,24.32,3.71,"56,265.00",2023-04-12,"1,317,613.73"
CNDA,10.34,10.75,3.81,"102,724.00",2023-05-02,"1,062,166.18"
NGMS,26.70,27.77,3.84,"31,388.00",2023-05-18,"838,059.62"
LYB,97.36,101.30,3.89,"896,512.00",2023-02-13,"87,284,408.87"
ACM,87.65,92.16,4.89,"366,594.00",2023-02-16,"32,131,964.66"


# Testing out (manually backtesting)

In [32]:
def get_info_test(df):        
    days_back = 12
    
    _highs = df['High'][:-days_back].max()
    _closes = df['Close'].iloc[-days_back]
    _pct_to_52 = ((_highs - _closes) / _highs) * 100
    # _volume = np.nanmean(df["Volume"].iloc[-days_back])
    _date_of_high = df['High'][:-days_back].idxmax()
    
    highs = df['High'].max()
    date_of_high = df['High'].idxmax()
    closes = df['Close'].iloc[-1]
    volume = df["Volume"].iloc[-3:].mean()  # Calculates the average volume in the last 3 days
    pct_to_52 = ((highs - closes) / highs) * 100
    
    new_df = pd.DataFrame({'Old Close': _closes, 'Old 52wk High': _highs, "Old Percent from 52": _pct_to_52, "Volume": volume, "doh": _date_of_high, "Close": closes, "Prct252": pct_to_52, "high52": highs, "date of break": date_of_high})

    return new_df

In [33]:
df_test = get_info_test(data)
cop_test = df_test.copy()
cop_test

,Old Close,Old 52wk High,Old Percent from 52,Volume,doh,Close,Prct252,high52,date of break
A,127.62,160.26,20.37,"1,333,326.33",2022-12-13,126.87,20.83,160.26,2022-12-13
AA,33.38,58.24,42.68,"3,132,267.33",2022-08-26,33.80,41.96,58.24,2022-08-26
AAC,10.56,10.60,0.38,"21,163.33",2023-07-18,10.65,0.05,10.65,2023-07-31
AACG,1.39,2.64,47.35,"2,835.00",2022-08-18,1.32,50.00,2.64,2022-08-18
AACI,10.51,12.20,13.85,"8,276.50",2023-01-30,NaN,NaN,12.20,2023-01-30
...,...,...,...,...,...,...,...,...,...
ZVSA,0.20,25.00,99.21,"1,000,905.00",2022-12-12,0.17,99.33,25.00,2022-12-12
ZWS,27.33,31.77,13.98,"590,255.00",2022-08-18,30.21,4.91,31.77,2022-08-18
ZYME,7.87,10.80,27.13,"923,709.67",2023-01-19,7.16,33.66,10.80,2023-01-19
ZYNE,0.36,1.39,74.10,"96,968.00",2022-08-08,0.35,74.97,1.39,2022-08-08


### Filtering and sorting

In [7]:
df_test = cop_test.copy()
time_thresh = datetime.now() - timedelta(days=120)
break_time_thresh = datetime.now() - timedelta(days=2)  # How many days ago it broke
vol_thresh = 10000
# print(break_time_thresh)
df_test = df_test[df_test["Old 52wk High"] != df_test["high52"]]
df_test = df_test[df_test["doh"] < time_thresh]
df_test = df_test[df_test["date of break"] > break_time_thresh]
df_test["diff"] = datetime.now() - df_test["date of break"]
pd.options.display.float_format = '{:.2f}'.format
df_test.sort_values(by='Volume', ascending=False).head(20)

NameError: name 'cop_test' is not defined

# Random shit

In [9]:
all_tickers = web.get_nasdaq_symbols()
all_tickers = all_tickers[~all_tickers["ETF"] & ~all_tickers["Test Issue"]]
# all_tickers.loc["O"]
all_tickers

,Nasdaq Traded,Security Name,Listing Exchange,Market Category,ETF,Round Lot Size,Test Issue,Financial Status,CQS Symbol,NASDAQ Symbol,NextShares
Symbol,,,,,,,,,,,
A,True,"Agilent Technologies, Inc. Common Stock",N,,False,100.00,False,NaN,A,A,False
AA,True,Alcoa Corporation Common Stock,N,,False,100.00,False,NaN,AA,AA,False
AAC,True,Ares Acquisition Corporation Class A Ordinary ...,N,,False,100.00,False,NaN,AAC,AAC,False
AAC.U,True,"Ares Acquisition Corporation Units, each consi...",N,,False,100.00,False,NaN,AAC.U,AAC=,False
AAC.W,True,Ares Acquisition Corporation Redeemable Warran...,N,,False,100.00,False,NaN,AAC.WS,AAC+,False
...,...,...,...,...,...,...,...,...,...,...,...
ZVSA,True,"ZyVersa Therapeutics, Inc. - Common Stock",Q,G,False,100.00,False,D,NaN,ZVSA,False
ZWS,True,Zurn Elkay Water Solutions Corporation Common ...,N,,False,100.00,False,NaN,ZWS,ZWS,False
ZYME,True,Zymeworks Inc. - Common Stock,Q,Q,False,100.00,False,N,NaN,ZYME,False


# Import to TradingView using pyautgui

In [6]:
def get_fit_tickers(df, num):
    return df.index[:num].tolist()


def add_to_tv(df):
    # Click plus (1773, 164)
    tickers = get_fit_tickers(df, 100)
    print(tickers)
    pyautogui.click(1773, 164)
    for ticker in tickers:
        pyautogui.write(ticker)
        pyautogui.press("enter")

# Finding stocks that broke 52wk high yesterday

### First DataFrame that contains necessary information

In [7]:
def parse_df(df, old_high=2, high_by="High"):
    # old_high- How many days ago it broke high (x or less)
    # high_by- by which OHLC to determine d
    high = df['High'].max()  # 52 wk high
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((high - close) / high) * 100  # Percent change to 52wk high
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['High'][:-old_high].idxmax()  # 52wk high before 2 days
    doh = df['High'].idxmax()  # 52wk high today
    new_df = pd.DataFrame({'Close': close, '52wk High': high, "% to 52wk": pct_to_52, "Volume": volume, "Date of high": doh, "Old date of high": old_doh, "Volume$": volume * close})

    return new_df

### Filtering and sorting DataFrame

In [8]:
def filter_sort_df(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a high volume but due to low price they have a high volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the high between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of high"] < time_thresh1) & (df["Old date of high"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of high"] != df["Date of high"]]
    
    return df.sort_values(by='% to 52wk', ascending=True)

In [9]:
full_df = parse_df(data, 2)
cop = full_df.copy()

df = filter_sort_df(cop)
df

,Close,52wk High,% to 52wk,Volume,Date of high,Old date of high,Volume$
DCBO,44.17,44.39,0.50,"52,000.00",2023-08-22,2023-04-03,"2,296,839.90"
REGN,840.63,847.50,0.81,"765,900.00",2023-08-22,2023-04-05,"643,838,520.74"
ACAC,10.89,11.11,1.98,"307,000.00",2023-08-22,2023-05-26,"3,343,230.11"
PAR,39.53,40.81,3.14,"373,600.00",2023-08-22,2023-02-14,"14,768,407.54"
PRLH,10.65,11.00,3.18,"267,700.00",2023-08-21,2023-02-13,"2,851,004.90"
APT,4.46,4.70,5.11,"329,500.00",2023-08-22,2023-02-07,"1,469,570.01"
GHLD,12.41,13.41,7.44,"43,300.00",2023-08-21,2023-02-23,"537,352.99"
FLXS,20.90,24.96,16.27,"56,100.00",2023-08-21,2023-03-21,"1,172,489.98"


In [16]:
add_to_tv(df)

['NEWR', 'ARGO', 'CHKP', 'LOCC', 'IVCA', 'CAE', 'CNDA', 'NGMS', 'LYB', 'ACM', 'ENSG', 'MNST', 'E', 'EDR', 'RWOD']


In [11]:
cop.loc["URI"]

Close                               NaN
52wk High                        492.22
% to 52wk                           NaN
Volume                              NaN
Date of high        2023-08-07 00:00:00
Old date of high    2023-08-07 00:00:00
Volume$                             NaN
Name: URI, dtype: object

# Finding low 52

In [25]:
def parse_df_low(df, old_low=2, low_by="Low"):
    # old_low- How many days ago it broke low (x or less)
    # low_by- by which OHLC to determine d
    low = df['Low'].min()  # 52 wk low
    close = df['Close'].iloc[-1]  # Close of last candle
    pct_to_52 = ((low - close) / low) * 100  # Percent change to 52wk low
    volume = df["Volume"].iloc[-1]  # Volume of last candle
    old_doh = df['Low'][:-old_low].idxmin()  # 52wk low before 2 days
    doh = df['Low'].idxmin()  # 52wk low today
    new_df = pd.DataFrame({'Close': close, '52wk Low': low, "% to 52wk": pct_to_52, "Volume": volume, "Date of low": doh, "Old date of low": old_doh, "Volume$": volume * close})

    return new_df

def filter_sort_df_low(df):
    df = df.dropna(subset=['Close'])

    vol_thresh = 10000  # Volume threshold
    vold_thresh = 500000  # Volume in dollar threshold, helps filtering out stocks that don't really have a low volume but due to low price they have a low volume
    time_thresh1 = datetime.now() - timedelta(days=75)  # Hit the low between 120 and 200 days ago (3 - 7 months)
    time_thresh2 = datetime.now() - timedelta(days=200)  

    df = df[df['Volume'] >= vol_thresh]
    df = df[df['Volume$'] >= vold_thresh]
    df = df[(df["Old date of low"] < time_thresh1) & (df["Old date of low"] > time_thresh2)]  # Old doh (last support) is between 90 to 200 days'
    df = df[df["Old date of low"] != df["Date of low"]]
    
    # return df.sort_values(by='% to 52wk', ascending=True)
    return df.sort_values(by='Date of low', ascending=True)

In [26]:
full_df = parse_df_low(data, 12)
cop = full_df.copy()

df = filter_sort_df_low(cop)
df

,Close,52wk Low,% to 52wk,Volume,Date of low,Old date of low,Volume$
ADTN,8.17,7.00,-16.68,"309,340.00",2023-08-07,2023-05-09,"2,526,534.31"
BNTX,123.35,95.50,-29.16,"1,124,301.00",2023-08-07,2023-05-11,"138,682,526.63"
CRON,1.78,1.64,-8.54,"957,938.00",2023-08-08,2023-06-06,"1,705,129.61"
AGEN,1.47,1.18,-24.58,"1,743,553.00",2023-08-08,2023-04-12,"2,563,022.96"
G,36.17,35.31,-2.44,"662,809.00",2023-08-08,2023-05-11,"23,973,800.32"
...,...,...,...,...,...,...,...
OCGN,0.40,0.40,-1.44,"7,020,635.00",2023-08-22,2023-05-26,"2,827,209.76"
CUTR,11.06,10.54,-4.93,"258,079.00",2023-08-22,2023-05-12,"2,854,353.85"
ONTF,6.80,6.60,-3.11,"240,340.00",2023-08-22,2023-05-22,"1,635,513.66"
CTVA,49.28,49.24,-0.10,"1,799,940.00",2023-08-22,2023-06-01,"88,710,042.63"
